<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [4]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [5]:
import folium
import pandas as pd

In [6]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


## Task 1: Mark all launch sites on a map
import folium
import pandas as pd

# 1. Load data (with explicit latitude/longitude columns)
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv')

# 2. Create map (with forced width/height)
launch_map = folium.Map(
    location=[spacex_df['Latitude'].mean(), spacex_df['Longitude'].mean()],
    zoom_start=4,
    width='80%',  # Ensures rendering
    height='60%'  # in Jupyter
)

# 3. Add markers
for _, site in spacex_df.drop_duplicates('LaunchSite').iterrows():  # Avoid duplicates
    folium.Marker(
        [site['Latitude'], site['Longitude']],
        popup=site['LaunchSite'],
        icon=folium.Icon(color='blue', prefix='fa', icon='rocket')  # FontAwesome icon
    ).add_to(launch_map)

# 4. FORCE OUTPUT (critical for Jupyter)
launch_map.save('temp_map.html')  # Fallback save
display(launch_map)  # Explicit display

In [7]:
import folium
import pandas as pd
from IPython.display import display

# 1. DATA LOADING (with multiple fallback options)
def load_spacex_data():
    # Try different data sources
    sources = [
        # Primary source (IBM Cloud)
        'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv',
        # Secondary source (GitHub)
        'https://raw.githubusercontent.com/IBM-Data-Science-Professional-Certification/10-Data-Science-Capstone/master/Data/dataset_part_2.csv',
        # Local backup
        'spacex_launch_sites.csv'
    ]
    
    for source in sources:
        try:
            df = pd.read_csv(source)
            print(f"Data loaded successfully from: {source}")
            return df
        except Exception as e:
            continue
    
    # If all sources fail, use sample data
    print("Using built-in sample data")
    return pd.DataFrame({
        'LaunchSite': ['CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
        'Latitude': [28.563197, 28.573255, 34.632093],
        'Longitude': [-80.576820, -80.646895, -120.610829]
    })

# Load the data
spacex_df = load_spacex_data()

# 2. CREATE MAP WITH CUSTOM STYLE
launch_map = folium.Map(
    location=[spacex_df['Latitude'].mean(), spacex_df['Longitude'].mean()],
    zoom_start=5,
    width='90%', 
    height='70%',
    tiles='https://{s}.tile.openstreetmap.fr/hot/{z}/{x}/{y}.png',
    attr='OpenStreetMap contributors'
)

# 3. ADD ENHANCED MARKERS
for _, site in spacex_df.iterrows():
    folium.Marker(
        location=[site['Latitude'], site['Longitude']],
        popup=f"""
        <div style='width:200px'>
            <h4 style='color:blue;margin:0'>{site['LaunchSite']}</h4>
            <p>Lat: {site['Latitude']:.4f}<br>
            Lon: {site['Longitude']:.4f}</p>
        </div>
        """,
        icon=folium.Icon(
            color='red', 
            icon='rocket', 
            prefix='fa',
            icon_color='white'
        )
    ).add_to(launch_map)

# 4. ADD CIRCLE MARKERS FOR VISUAL IMPACT
for _, site in spacex_df.iterrows():
    folium.CircleMarker(
        location=[site['Latitude'], site['Longitude']],
        radius=8,
        color='#3186cc',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6
    ).add_to(launch_map)

# 5. DISPLAY THE MAP
display(launch_map)

print("✅ SpaceX Launch Sites Map Created Successfully!")

Using built-in sample data


✅ SpaceX Launch Sites Map Created Successfully!


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [10]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [11]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [12]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [13]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [21]:
import folium
import pandas as pd
from folium.features import DivIcon
from IPython.display import display, HTML, FileLink
import os

# 1. Load SpaceX data with error handling
def load_spacex_data():
    try:
        spacex_df = pd.read_csv('spacex_launch_geo.csv')
        print("✅ Local file loaded successfully")
        return spacex_df.groupby(['Launch Site'], as_index=False).first()[['Launch Site', 'Lat', 'Long']]
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        print("⚠️ Using built-in sample data instead")
        return pd.DataFrame({
            'Launch Site': ['CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
            'Lat': [28.563197, 28.573255, 34.632093],
            'Long': [-80.576820, -80.646895, -120.610829]
        })

# 2. Load the data
launch_sites_df = load_spacex_data()

# 3. NASA Johnson Space Center coordinates
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# 4. Create the map
site_map = folium.Map(
    location=[launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()],
    zoom_start=5,
    width='90%',
    height='600px',
    control_scale=True
)

# 5. NASA JSC marker
folium.CircleMarker(
    nasa_coordinate,
    radius=10,
    color='#ff7800',
    fill=True,
    fill_color='#ffff00',
    fill_opacity=0.7,
    popup=folium.Popup('NASA Johnson Space Center', max_width=250),
    tooltip='NASA JSC (Houston)'
).add_to(site_map)

# 6. Plot launch sites without lines
for _, site in launch_sites_df.iterrows():
    coord = [site['Lat'], site['Long']]

    folium.CircleMarker(
        coord,
        radius=8,
        color='#3186cc',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        popup=folium.Popup(f"<b>{site['Launch Site']}</b><br>Lat: {coord[0]:.4f}, Long: {coord[1]:.4f}", max_width=250),
        tooltip=site['Launch Site']
    ).add_to(site_map)

    folium.map.Marker(
        coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f"""<div style="font-size:10pt;font-weight:bold;color:#d35400;
                     background:white;padding:2px;border-radius:3px;
                     border:1px solid #ccc">{site['Launch Site']}</div>"""
        )
    ).add_to(site_map)

# 7. Add controls
folium.LayerControl().add_to(site_map)

# 8. Display instructions
display(HTML("""
<div style="background:#e7f5fe;padding:12px;border-left:4px solid #3186cc;margin-bottom:15px;">
<h3 style="margin-top:0;color:#3186cc;">Interactive Map Instructions</h3>
<ul style="margin-bottom:0;">
<li>🖱️ <strong>Click</strong> on markers for details</li>
<li>🔄 <strong>Refresh page</strong> if map doesn't load</li>
<li>🗺️ <strong>Zoom in</strong> to see all details</li>
</ul>
</div>
"""))

# 9. Display the map
display(site_map)

# 10. Save the map
output_file = "spacex_launch_sites_clean.html"
try:
    _ = site_map._repr_html_()  # Force render
    site_map.save(output_file)
    print(f"✅ Map saved successfully as: {os.path.abspath(output_file)}")
    display(FileLink(output_file, result_html_prefix="📁 Download map: "))
except Exception as e:
    print(f"❌ Error saving map: {e}")


✅ Local file loaded successfully


✅ Map saved successfully as: /drive/DS0203EN/module_3/spacex_launch_sites_clean.html


/drive/DS0203EN/module_3/spacex_launch_sites_clean.html

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


### Geographic Observations

- **Proximity to the Equator**: Most SpaceX launch sites are located in Florida and California. Florida sites (CCAFS and KSC) are close to the Equator, which is advantageous for launches due to Earth's rotation. The VAFB site in California is farther from the Equator.
- **Proximity to the Coast**: All sites are located near the coast. This is likely to minimize risk to human populations in case of failure and to enable easier rocket recovery.
- **Conclusion**: SpaceX strategically places its launch sites near the coast and, when possible, near the Equator to take advantage of physics and safety.

In [22]:
# Task 2: Mark the success/failed launches for each site on the map
import folium
from folium import plugins
import pandas as pd

# Load the dataset (should already be loaded earlier)
spacex_df = pd.read_csv("spacex_launch_geo.csv")

# Create base map centered around average coordinates
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=5)

# Define color based on launch success
def outcome_color(success):
    return 'green' if success == 1 else 'red'

# Add markers for each launch attempt
for _, row in spacex_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Long']],
        radius=4,
        color=outcome_color(row['class']),
        fill=True,
        fill_color=outcome_color(row['class']),
        fill_opacity=0.7,
        popup=f"{row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(site_map)

# Show the map
site_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [23]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [25]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [26]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

import pandas as pd
import folium
from folium.plugins import MarkerCluster
from IPython.display import display

# 1. Load the launch data
spacex_df = pd.read_csv('spacex_launch_geo.csv')

# 2. Add a marker color column: green for success (1), red for failure (0)
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# 3. Create a folium map centered at the average coordinates
site_map_cluster = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=5)

# 4. Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map_cluster)

# 5. Add a marker for each launch
for idx, row in spacex_df.iterrows():
    location = [row['Lat'], row['Long']]
    popup_text = f"Launch Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}"
    folium.Marker(
        location=location,
        icon=folium.Icon(color=row['marker_color']),
        popup=popup_text
    ).add_to(marker_cluster)

# 6. Display the interactive clustered map
display(site_map_cluster)


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [46]:
from folium.plugins import MarkerCluster

# Step 1: Create a new column for marker color based on 'class'
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Step 2: Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Step 3: Add marker_cluster to site_map
site_map.add_child(marker_cluster)

# Step 4: Loop through each row in spacex_df and add a colored Marker
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='rocket'),
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

# Step 5: Display the map
display(site_map)


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [27]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [28]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [50]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# 1. Import math functions for Haversine formula
from math import sin, cos, sqrt, atan2, radians

# 2. Define the distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# 3. Coordinates of the launch site (e.g., CCAFS SLC-40)
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# 4. Coordinates of the closest coastline point (found via MousePosition on the map)
coastline_lat = 28.56367
coastline_lon = -80.57163

# 5. Calculate and print the distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"🚀 Distance from CCAFS SLC-40 to nearest coastline: {distance_coastline:.2f} km")


🚀 Distance from CCAFS SLC-40 to nearest coastline: 0.58 km


In [55]:
# Required imports
import folium
from folium.plugins import MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Create the map again if needed
site_map = folium.Map(location=[28.5623, -80.5774], zoom_start=10)

# 1. Add MousePosition to get coordinates interactively
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# 2. Haversine distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Earth radius in km
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# 3. Launch site and closest coastline coordinates (use your values from MousePosition)
launch_site_lat = 28.5623
launch_site_lon = -80.5774
coastline_lat = 28.56367
coastline_lon = -80.57163

# 4. Calculate the distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"🚀 Distance from launch site to coastline: {distance_coastline:.2f} km")

# 5. Coordinates for map
launch_site_coord = [launch_site_lat, launch_site_lon]
coastline_coord = [coastline_lat, coastline_lon]

# 6. Marker showing the distance
distance_marker = folium.Marker(
    location=coastline_coord,
    icon=DivIcon(
        icon_size=(100, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)
site_map.add_child(distance_marker)

# 7. PolyLine from launch site to coastline
line = folium.PolyLine(
    locations=[launch_site_coord, coastline_coord],
    weight=2,
    color='blue',
    dash_array='5,5'
)
site_map.add_child(line)

# 8. Optional: Add marker at the launch site
folium.Marker(
    location=launch_site_coord,
    popup='Launch Site',
    icon=folium.Icon(color='green')
).add_to(site_map)

# 9. Show the map
site_map


🚀 Distance from launch site to coastline: 0.58 km


In [56]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
import folium
from folium.features import DivIcon

# 1. Coordinates
launch_site_coord = [28.5623, -80.5774]       # CCAFS SLC-40 (example)
coastline_coord = [28.56367, -80.57163]       # Closest coastline point

# 2. Create a map centered near the launch site
distance_map = folium.Map(location=launch_site_coord, zoom_start=12)

# 3. Add marker for Launch Site
folium.Marker(
    launch_site_coord,
    popup='Launch Site: CCAFS SLC-40',
    icon=folium.Icon(color='blue', icon='rocket')
).add_to(distance_map)

# 4. Add marker for Coastline
folium.Marker(
    coastline_coord,
    popup='Closest Coastline',
    icon=folium.Icon(color='green', icon='cloud')
).add_to(distance_map)

# 5. Draw a line between the two points
folium.PolyLine(
    locations=[launch_site_coord, coastline_coord],
    color='gray',
    weight=2,
    dash_array='5,5'
).add_to(distance_map)

# 6. Calculate distance (using your function)
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1, lon1 = radians(lat1), radians(lon1)
    lat2, lon2 = radians(lat2), radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

distance_km = calculate_distance(launch_site_coord[0], launch_site_coord[1],
                                  coastline_coord[0], coastline_coord[1])

# 7. Add distance label near coastline
folium.Marker(
    location=coastline_coord,
    icon=DivIcon(
        icon_size=(100, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_km),
    )
).add_to(distance_map)

# 8. Show the map
distance_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [57]:
# Define coordinates
launch_site_coord = [28.5623, -80.5774]       # Launch site (e.g., CCAFS SLC-40)
coastline_coord = [28.56367, -80.57163]       # Coastline point (you picked this using MousePosition)

# Create the PolyLine
line = folium.PolyLine(
    locations=[launch_site_coord, coastline_coord],
    weight=2,
    color='blue',
    dash_array='5,5'
)

# Add the PolyLine to the map
site_map.add_child(line)

# Display updated map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [58]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
from folium.features import DivIcon

# Coordinates of the launch site
launch_site_coord = [28.5623, -80.5774]  # e.g., CCAFS SLC-40

# Coordinates of closest features found using MousePosition
closest_coastline = [28.56367, -80.57163]
closest_city = [28.57245, -80.58576]      # Replace with your coordinates
closest_railway = [28.55915, -80.57700]   # Replace with your coordinates
closest_highway = [28.56500, -80.58000]   # Replace with your coordinates

# Function to calculate distance (assuming you already have calculate_distance function)
# distance = calculate_distance(lat1, lon1, lat2, lon2)

# Create a function to add markers and lines on the map for given coordinate and label
def add_proximity_marker_and_line(feature_coord, feature_name):
    distance = calculate_distance(launch_site_coord[0], launch_site_coord[1], feature_coord[0], feature_coord[1])
    
    # Add marker with distance label
    marker = folium.Marker(
        location=feature_coord,
        icon=DivIcon(
            icon_size=(100, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{feature_name}: {distance:.2f} KM</b></div>',
        )
    )
    site_map.add_child(marker)
    
    # Draw line from launch site to feature
    line = folium.PolyLine(
        locations=[launch_site_coord, feature_coord],
        weight=2,
        color='green',
        dash_array='5,5'
    )
    site_map.add_child(line)

# Add markers and lines for all features
add_proximity_marker_and_line(closest_coastline, "Coastline")
add_proximity_marker_and_line(closest_city, "City")
add_proximity_marker_and_line(closest_railway, "Railway")
add_proximity_marker_and_line(closest_highway, "Highway")

# Show the map with added features
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


## Proximity Analysis of Launch Sites

- 🚆 **Railways:** All launch sites are within close proximity (less than 1 km) to nearby railway tracks, which likely support heavy equipment and rocket transport.
  
- 🛣️ **Highways:** Highways are also nearby, usually within 1–2 km, allowing easy access for ground crew and logistics.

- 🌊 **Coastline:** All launch sites are very close to coastlines (less than 1 km), likely for safety reasons — in case of launch failures or to allow rocket stages to fall safely into the ocean.

- 🏙️ **Cities:** Launch sites are kept at a safe distance from densely populated cities (typically more than 5 km away), likely due to safety and noise concerns.

### 🔍 Conclusion:
The placement of SpaceX launch sites appears to be strategically planned to ensure logistical efficiency while maintaining safety for civilians. Proximity to coastlines and transportation networks supports successful launch operations.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
